In [1]:
import dask.array as da
import dask.bag as db
from dask.dataframe import Series
import dask.dataframe as dd
import warnings
from sklearn.feature_extraction.text import TfidfTransformer
from dask.distributed import Client, TimeoutError
import scipy as sp
import scipy.sparse as sps
from numba.typed import Dict, List
from numba import prange
from numba.types import unicode_type
from scipy.sparse import csr_matrix
import pandas as pd
import glob
import re
import io
import numpy as np
from collections import Counter
import dask.dataframe as dd
import numba as nb
from glob import glob
import pyximport;pyximport.install(language_level = '3',
                                  setup_args={"include_dirs":np.get_include()})
import cythongenomeanalyzer as cyGA
import umap
import time
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances
import math
import numba as nb
from numba.typed import Dict
from numba import types
import random



C:\Users\thiag\anaconda3\envs\UMAP\lib\site-packages\umap_learn-0.5.0rc1-py3.8.egg\umap\__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


In [2]:
##### basic numba functions

@nb.njit(fastmath = True)
def char2int(chars):
    if chars == 65: return 0
    elif chars == 67: return 1
    elif chars == 71: return 2
    elif chars == 84: return 3
    return -1

@nb.njit(fastmath = True)
def invchar2int(chars):
    if chars == 65: return 3
    elif chars == 67: return 2
    elif chars == 71: return 1
    elif chars == 84: return 0
    return -1

@nb.njit(fastmath = True)
def str2int(chars):
    if chars == 'A' or chars == 'a': return 0
    elif chars == 'C' or chars == 'c': return 1
    elif chars == 'G' or chars == 'g': return 2
    elif chars == 'T' or chars == 't': return 3
    return -1

@nb.njit(fastmath = True)
def invstr2int(chars):
    if chars == 'A' or chars == 'a': return 3
    elif chars == 'C' or chars == 'c': return 2
    elif chars == 'G' or chars == 'g': return 1
    elif chars == 'T' or chars == 't': return 0
    return -1


@nb.njit(fastmath = True)
def _numba_dict_fastq(file_mmap, klen, mg_fw_rv = True): 
    fw_div = 4**(klen-1)
    is_line = 1 
    window_pos = 0
    fw_num = 0
    rv_num = 0
    result =  Dict.empty(key_type=types.uint64,value_type=types.uint32)
    if mg_fw_rv == True:
        for c in file_mmap:
            if c == 78: window_pos, fw_num, rv_num = 0,0,0
            elif c == 13: pass
            elif c == 10: 
                is_line += 1
                if is_line > 4: is_line= 1
                window_pos = 0
            elif is_line == 2: 
                fw_num = fw_num*4 + char2int(c)
                rv_num = invchar2int(c)*(4**window_pos) + rv_num
                window_pos += 1
                if window_pos == klen: 
                    if fw_num < rv_num: 
                        try: result[fw_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    else: 
                        try: result[rv_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    fw_num %= fw_div
                    rv_num //= 4
                    window_pos -= 1
    else:
        for c in file_mmap:
            if c == 78: window_pos, fw_num = 0,0
            elif c == 13: pass
            elif c == 10: 
                is_line += 1
                if is_line > 4: is_line= 1
                window_pos = 0
            elif is_line == 2: 
                fw_num = fw_num*4 + char2int(c)
                window_pos += 1
                if window_pos == klen: 
                    try: result[fw_num] += types.int32(1)
                    except: result[fw_num] = types.int32(1)
                    
                    fw_num %= fw_div
                    window_pos -= 1
    return result

@nb.njit(fastmath = True)
def _numba_dict_fasta(file_mmap, klen, mg_fw_rv = True): 
    fw_div = 4**(klen-1)
    window_pos = 0
    fw_num = 0
    rv_num = 0
    result =  Dict.empty(key_type=types.int64,value_type=types.uint32)
    header_end = 0
    if file_mmap[0] == 62: 
        while file_mmap[header_end] != 10: header_end+=1 
            
    if mg_fw_rv == True:
        for c in file_mmap[header_end:]:
            if c == 78: window_pos, fw_num, rv_num = 0,0,0
            elif c == 10 or c == 13: pass
            else: 
                fw_num = fw_num*4 + char2int(c)
                rv_num = invchar2int(c)*(4**window_pos) + rv_num
                window_pos += 1
                if window_pos == klen: 
                    if fw_num < rv_num: 
                        try: result[fw_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    else: 
                        try: result[rv_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    fw_num %= fw_div
                    rv_num //= 4
                    window_pos -= 1
    else:
        for c in file_mmap[header_end:]:
            if c == 78: window_pos, fw_num = 0,0
            elif c == 10 or c == 13: pass
            else: 
                fw_num = fw_num*4 + char2int(c)
                window_pos += 1
                if window_pos == klen: 
                    try: result[fw_num] += types.uint32(1)
                    except: result[fw_num] = types.uint32(1)
                    fw_num %= fw_div
                    window_pos -= 1
    return result

@nb.njit(fastmath = True)
def _numba_dict_string(string, klen,  mg_fw_rv = True): 
    fw_div = 4**(klen-1)
    window_pos = 0
    fw_num = 0
    rv_num = 0
    result =  Dict.empty(key_type=types.uint64,value_type=types.uint32)
    if mg_fw_rv:
        for c in string:
            if c == 'N': window_pos, fw_num, rv_num = 0,0,0
            elif c == '\n'  or c == '\r': pass
            else: 
                fw_num = fw_num*4 + str2int(c)
                rv_num = invstr2int(c)*(4**window_pos) + rv_num
                window_pos += 1
                if window_pos == klen: 
                    if fw_num < rv_num:
                        try: result[fw_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    else: 
                        try: result[rv_num] += types.uint32(1)
                        except: result[fw_num] = types.uint32(1)
                    fw_num %= fw_div
                    rv_num //= 4
                    window_pos -= 1
    else:
        for c in string:
            if c == 'N': window_pos, fw_num = 0,0
            elif c == '\n' or c == '\r': pass
            else: 
                fw_num = fw_num*4 + str2int(c)
                window_pos += 1
                if window_pos == klen: 
                    try: result[fw_num] += types.uint32(1)
                    except: result[fw_num] = types.uint32(1)
                    fw_num %= fw_div
                    window_pos -= 1
    return result

def dic2sparserow(dic, maxsize =  int(1e8)):
    return sps.csr_matrix((list(dic.values()), list(dic.keys()), [0, len(dic)]), shape = (1, maxsize ))


#@nb.jit
def nb_multiple_files(filenames = [], klen = 10, cnt = 'auto', merge_reverse_complement = True):
    max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
    if cnt == 'auto':
        typ = filenames[0].split('.')[-1]
        if typ == filenames[0]: typ = 'string'
    else: typ = cnt    
    if typ == 'fastq': 
        ret = sps.vstack((dic2sparserow(_numba_dict_fastq(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
    elif typ == 'fasta' or typ == 'fa':
        ret = sps.vstack((dic2sparserow(_numba_dict_fasta(name, klen, merge_reverse_complement), max_size) for  name  in filenames))
    elif typ == 'string':
        ret = sps.vstack((dic2sparserow(_numba_dict_string(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
    else:
        ret = sps.eye(0, dtype = np.uint32 ,format="csr")
    ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
    return ret

def nb_sparse_fasta(filenames = [], klen = 10, merge_reverse_complement = True):
    max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
    ret = sps.vstack((dic2sparserow(_numba_dict_fasta(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
    ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
    return ret

def nb_sparse_string(filenames = [], klen = 10,  merge_reverse_complement = True):
    max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
    ret = sps.vstack((dic2sparserow(_numba_dict_string(name, klen, merge_reverse_complement), max_size) for  name  in filenames))
    ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
    return ret

def nb_sparse_fastq(filenames = [], klen = 10, merge_reverse_complement = True):
    max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
    ret = sps.vstack((dic2sparserow(_numba_dict_fastq(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
    ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
    return ret

In [4]:
%time aaaa2 = nb_sparse_string([''.join(random.choices('ATCG', k=200)) for i in range(10000)], 30, False)
        


Wall time: 6.26 s


In [3]:
class KmerGenomeAnalyser:
    
    def __init__(self,minorallelefrequency = 0, kmer_size = 10, MergeReverseComplement = False):
        if minorallelefrequency:
            if type(minorallelefrequency) == int or type(minorallelefrequency) == float:
                self.maf = sorted([minorallelefrequency, 1-minorallelefrequency])
            elif type(minorallelefrequency) == list or type(minorallelefrequency) == tuple:
                self.maf = sorted([minorallelefrequency[0], minorallelefrequency[1]])
            else:
                raise ValueError('Minor Allele Fequency not accepted')
        else: self.maf =0
        
        if kmer_size < 0:
            raise ValueError('Kmer size has to be positive')
        else: self.kmer = kmer_size
        
        self.revComp = MergeReverseComplement
        self.n_docs = 0
                
    def nbG2S(self, files = [], tfidf = False, binary_output = False, content = 'auto', single_row_output = False):
        ncols = 4**self.kmer - 4**(self.kmer//2) if self.revComp else 4**self.kmer
        if content == 'auto':
            if (typ := files[0].split('.')[-1]) == files[0]: typ = 'string'
        else: typ = content
        
        b = db.from_sequence(files)
        if typ == 'cython':
            #raise ValueError('Windows and Cython are not friends')
            bag2 = b.map_partitions(cyGA.cyfiles2sparse, klen = self.kmer , merge_reverse_complement = self.revComp )
        elif typ == 'fastq':
            bag2 = b.map_partitions(self.nb_sparse_fastq, klen = self.kmer , merge_reverse_complement = self.revComp )
        elif typ == 'fasta' or typ == 'fa':
            bag2 = b.map_partitions(self.nb_sparse_fasta, klen = self.kmer , merge_reverse_complement = self.revComp )
        elif typ == 'string':
            bag2 = b.map_partitions(self.nb_sparse_string, klen = self.kmer , merge_reverse_complement = self.revComp )
        else: return 'content type not found'

        objs = bag2.to_delayed()
        arrs = [da.from_delayed(obj, (np.nan, ncols), str) for obj in objs]
        ret = da.concatenate(arrs, axis=0).compute()
        
        self.n_docs = ret.shape[0]
                
        if self.maf:
            self.n_docs = ret.shape[0]
            idx = np.unique(ret.indices, return_counts=True)
            self.acceptedindices  =  idx[0][(self.maf[1]*self.n_docs >= idx[1])&(idx[1]>= self.maf[0]*self.n_docs)]
            if len(self.acceptedindices) == 0:  raise ValueError('No kmers present with minor allele frequency range sent')
            ret =  ret.tolil()[:,  self.acceptedindices ].tocsr()
        
        else:
            self.acceptedindices = np.unique(ret.indices)
        
        if tfidf: ret =  TfidfTransformer().fit_transform(ret)
        if binary_output: ret.data.fill(1)
        if single_row_output: 
            ret = sps.csr_matrix((ret.data, ret.indices, (0, ret.indptr[-1])), dtype = np.int16, copy=False, shape = (1, ncols))
            self.n_docs = 1
        del arrs, objs, bag2, b
        return ret
    
    def applymaf( X, maf):
        if type(maf) == int or type(maf) == float:mafed = sorted([maf, 1-maf])
        elif type(maf) == list or type(maf) == tuple: mafed = sorted([maf[0], maf[1]])
        else: raise ValueError('Minor Allele Fequency not accepted')
        warnings.warn("kmer IDs will be lost if <class>.maf is not altered too") 
        n_doc = X.shape[0]
        idx = np.unique(X.indices, return_counts=True)
        out = idx[0][(mafed[1]*n_doc >= idx[1])&(idx[1]>= mafed[0]*n_doc)]
        return X.tolil()[:, out].tocsr()

    def dic2sparserow(dic, maxsize =  int(1e8)):
        return sps.csr_matrix((list(dic.values()), list(dic.keys()), [0, len(dic)]), shape = (1, maxsize ))        

    def kmer_list(self, matrix = None):
        if matrix != None:
            return [self.int2kmer(i) for i in self.acceptedindices]
        else:     
            return [self.int2kmer(i) for i in self.acceptedindices]
        
    
    def int2kmer(self, i):
        return np.base_repr(i,4).replace('0', 'A').replace('1', 'C').replace('2', 'G').replace('3', 'T')
    
    @staticmethod
    @nb.njit(fastmath = True)
    def str_to_int(s):
        final_index, result = len(s) - 1, 0
        for i,v in enumerate(s):
            result += (ord(v) - 48) * (4 ** (final_index - i))
        return result

    def nb_sparse_fasta(self, filenames = [], klen = 10, merge_reverse_complement = True):
        max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
        ret = sps.vstack((dic2sparserow(_numba_dict_fasta(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
        ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
        return ret

    def nb_sparse_string(self,filenames = [], klen = 10,  merge_reverse_complement = True):
        max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
        ret = sps.vstack((dic2sparserow(_numba_dict_string(name, klen, merge_reverse_complement), max_size) for  name  in filenames))
        ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
        return ret

    def nb_sparse_fastq(self, filenames = [], klen = 10, merge_reverse_complement = True):
        max_size = 4**klen - 4**(klen//2) if merge_reverse_complement else 4**klen 
        ret = sps.vstack((dic2sparserow(_numba_dict_fastq(np.memmap(name,  mode='r'), klen, merge_reverse_complement), max_size) for  name  in filenames))
        ret._meta = sps.eye(0, dtype = np.uint32 ,format="csr")
        return ret
    
    def tfidf(X):
        return TfidfTransformer().fit_transform(X)

    def hellinger_dist_matrix(X):
        return pairwise_distances(np.sqrt(X), n_jobs=-1, metric='euclidean') / math.sqrt(2)
    
    @staticmethod
    @nb.njit(fastmath = True)
    def str_to_int(s):
        final_index, result = len(s) - 1, 0
        for i,v in enumerate(s):
            result += (ord(v) - 48) * (4 ** (final_index - i))
        return result
    
    def plot_clusters(data, algorithm, args, kwds):
        labels = algorithm(*args, **kwds).fit_predict(data)
        palette = sns.color_palette('deep', np.unique(labels).max() + 1)
        colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
        plt.scatter(data.T[0], data.T[1], c=colors, **{'alpha' : 0.25, 's' : 80, 'linewidths':0})
        frame = plt.gca()
        frame.axes.get_xaxis().set_visible(False)
        frame.axes.get_yaxis().set_visible(False)
        sns.despine()
    


In [6]:
from dask.distributed import Client

client = Client(processes = False)
client

Client Scheduler: inproc://192.168.3.87/14388/1 Dashboard: http://192.168.3.87:8787/status,Cluster Workers: 1 Cores: 12 Memory: 16.91 GB


In [7]:
df = pd.DataFrame(sorted(glob('covid_fasta/*.fasta')) , columns = ['filename'])
df['id'] = df.filename.apply(lambda x: x.strip('covid_fasta\\').strip('.fasta')[:-2]  )
df2 = pd.read_csv('covid_metadata_w_us.csv')
df3 = df.merge(df2, left_on= 'id', right_on='Accession')
df4 = df3.sample(frac=1)
df4.Collection_Date = df4.Collection_Date.apply(pd.to_datetime)

In [8]:
Analyser_class = KmerGenomeAnalyser(minorallelefrequency = 0, kmer_size = 20, MergeReverseComplement = False)

In [9]:
%time X = Analyser_class.nbG2S(df4[:1000].filename,  content ='fasta') # content = 'cython' int(1e8)
X

Wall time: 2min 36s


<1000x1099511627776 sparse matrix of type '<class 'numpy.float64'>'
	with 28236673 stored elements in Compressed Sparse Row format>

In [ ]:
%time Xsm = KmerGenomeAnalyser.applymaf(X, [.05,.95])

In [ ]:
KmerGenomeAnalyser.tfidf(Xsm)

# regex tokenizer kmer


In [122]:
list(set([1,2,3,1,12]))

[1, 2, 3, 12]

In [123]:
np.unique(np.array([1,3,4,]))

array([1, 3, 4])